In [96]:
import re
from importlib import resources

import tiktoken

from loguru import logger

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import load_prompt
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationSummaryBufferMemory, ChatMessageHistory

import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions

from bot import NewsBot
from bot import BotConfig


_config = BotConfig()

In [40]:
bot = NewsBot()

No sentence-transformers model found with name /home/jovyan/.cache/torch/sentence_transformers/unicamp-dl_ptt5-base-portuguese-vocab. Creating a new one with MEAN pooling.


In [41]:
bot.execute("Qual a última notícia de acidentes no bairro Dom Bosco?")



> Entering new LLMChain chain...
Prompt after formatting:
Voce vai receber uma pergunta de follow up do usuario, que pode nao ter muitos detalhes.
O usuario quer realizar consultas na base de dados que contem noticias de Pocos de Caldas e regiao.
REESCREVA a frase do HUMAN INPUT para que ela possa ser considerada uma pergunta independente.
Considere o CHAT HISTORY para manter o contexto.
NAO RESPONDA AO USUARIO, APENAS REESCREVA A PERGUNTA.
Sua resposta deve conter apenas a pergunta reescrita.

## CHAT HISTORY


## HUMAN INPUT
Qual a última notícia de acidentes no bairro Dom Bosco?

IA:


2024-01-05 02:39:11.078 | DEBUG    | bot._newsbot:execute:206 - Pergunta original: Qual a última notícia de acidentes no bairro Dom Bosco?
2024-01-05 02:39:11.079 | DEBUG    | bot._newsbot:execute:207 - Pergunta melhorada: Qual é a última notícia de acidentes no bairro Dom Bosco?



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Analise a mensagem do usuario em "HUMAN INPUT" e o "CHAT HISTORY" e classifique a intencao de acordo com as seguintes categorias:

- "Inicio de conversa": Saudacoes e cumprimentos do usuario, reconheca frases como "oi", "ola", "bom dia", "tudo bem:", etc.
- "Consulta de conteudo": Perguntas sobre noticias da base de dados, reconheca trechos como "quais sao as ultimas noticias", "o que foi reportado no bairro", etc.
- "": QUALQUER PERGUNTA QUE NAO SE ENCAIXE NAS CATEGORIAS ACIMA, ou que nao possa ser respondida com o contexto disponivel na base de dados, a qual contem noticias de Pocos de Caldas e regiao.

Escolha apenas uma das opcoes. Sua resposta deve conter APENAS a categoria.

## CHAT HISTORY


## HUMAN INPUT
Qual é a última notícia de acidentes no bairro Dom Bosco?

IA:


2024-01-05 02:39:12.021 | DEBUG    | bot._newsbot:execute:210 - Intenção: Consulta de conteúdo
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Voce e um chatbot que responde sobre noticias da cidade de Pocos de Caldas.

Sua tarefa e responder a mensagem do usuário em HUMAN INPUT utilizando o CONTEXTO providenciado.
Considere o CHAT HISTORY.
Nao responda nada que nao esteja no contexto. Se o contexto nao tiver informacoes relevantes a pergunta do usuario, responda "nao sei".
Nao considere nenhuma noticia que nao seja de Pocos de Caldas.

Formate sua resposta como um JSON conforme indicado abaxo:
{
    "resposta": "sua resposta aqui",
    "link": "link da noticia que voce usou para elaborar sua resposta",
    "data": "data da noticia",
    "titulo": "titulo da noticia",
    "autor": "autor da noticia"
}
Nao responda nada alem do JSON.

## CONTEXTO
<data>2019-11-17</data>
<titulo>Antes de ser lançada, decoração natalina é alvo de vandalismo em Poços</titulo>
<autor>ondapocos</autor>
<link>https://ondapocos.com.br/antes-de-ser-lancada-decoracao-natalin

2024-01-05 02:39:19.135 | INFO     | bot._newsbot:execute:228 - >>> Resposta: resposta: Não sei.

link_noticia: 

data_noticia: 

titulo_noticia: 

autor_noticia: 





> Finished chain.


{'response': 'resposta: Não sei.\n\nlink_noticia: \n\ndata_noticia: \n\ntitulo_noticia: \n\nautor_noticia: \n\n',
 'execution_id': '78d8d9a215b7401890fa3512e068e74c'}

# Functions

In [102]:
def get_prompt(key):

    logger.debug("Estou em 'get_prompt'")
    filepath = str(
        resources.files("bot.prompts").joinpath(f"{key}.json")
    )

    return load_prompt(filepath)

def self_get_chroma_collection():

    logger.debug("Estou em 'self_get_chroma_collection'")

    host_name = _config.VECTORDATABASE_HOSTNAME
    port = _config.VECTORDATABASE_PORT
    collection_name = _config.EMBEDDING_COLLECTION
    persist_directory = _config.VECTORDATABASE_PERSIST_DIRECTORY
    
    settings = Settings(allow_reset=True, anonymized_telemetry=True, persist_directory=persist_directory)
    chroma_client = chromadb.HttpClient(host=host_name, port = port, settings=settings)
    
    return chroma_client.get_collection(collection_name, embedding_function=self_embedder)


def self_format_history_message(messages, human_prefix: str = 'Human', ai_prefix: str = 'AI'):

    logger.debug("Estou em 'self_format_history_message'")
    for message in messages:
        if isinstance(message, langchain_core.messages.human.HumanMessage):
            yield f"{human_prefix}: {message.content}\n"
        
        elif isinstance(message, langchain_core.messages.ai.AIMessage):
            yield f"{ai_prefix}: {message.content}\n"

def self_get_standalone_question(message: str, *args, **kwargs) -> str:

    logger.debug("Estou em 'self_get_standalone_question'")

    self_chain_standalone_question = LLMChain(
        llm=self_llm, prompt=self_prompt_standalone_question, verbose=self_verbose_chains
    )

    return self_chain_standalone_question.predict(history=kwargs["history"], human_input=message)

def self_get_user_intention(message: str, *args, **kwargs) -> str:

    logger.debug("Estou em 'self_get_user_intention'")

    self_chain_intention = LLMChain(
        llm=self_llm, prompt=self_prompt_intention, verbose=self_verbose_chains
    )
    intention = self_chain_intention.predict(history=kwargs["history"], human_input=message)

    return intention.lower().replace("ú", "u").replace("í", "i")

def extract_dict_from_string(string):

    logger.debug("Estou em 'extract_dict_from_string'")

    match = re.search(r"{([^}]+)}", string)
    if match is None:
        return dict()

    json_content = match.group(1)

    try:
        json_dict = json.loads("{" + json_content + "}")
        return json_dict
    except json.JSONDecodeError as err:
        logger.error(str(err))
        return dict()

def self_predict(collection, query, n_neighbors: int = 1000, n_results: int = 10, **kwargs):

    logger.debug("Estou em 'self_predict'")

    def f(key, value, k):
        def limit(x):
            if isinstance(x, list):
                return x[:k]
            return x
        if isinstance(value, list):
            return (key, [limit(x) for x in value])
        return (key, None)
    
    
    res = collection.query(
        query_texts=query,
        n_results=n_neighbors,
        **kwargs
        # where={"metadata_field": "is_equal_to_this"},
        # where_document={"$contains":"search_string"}
    )
  
    return  dict([f(key, value, n_results)  for key, value in res.items()])

def self_count_tokens(context: str) -> int:

    logger.debug("Estou em 'self_count_tokens'")
    encoding = tiktoken.encoding_for_model(_config.LLM_MODEL_NAME)
    num_tokens = len(encoding.encode(context))
    
    return num_tokens

def self__set_query_content(documents, metadata):

    logger.debug("Estou em 'self__set_query_content'")
    context_list = [self__set_local_context(content, meta) for content, meta in zip(documents, metadata)]

    tokens = 4000
    max_tokens = 3600
    while tokens > max_tokens:
        tokens = self_count_tokens("\n".join(context_list))
        if tokens >= max_tokens:
            context_list.pop()

def self__set_local_context(content, meta):

    logger.debug("Estou em 'self__set_local_context'")
    return (
        f"<data>{meta['date']}</data>\n"
        f"<titulo>{meta['title']}</titulo>\n"
        f"<autor>{meta['author']}</autor>\n"
        f"<link>{meta['link']}</link>\n"
        f"<conteudo>{content}</conteudo>\n"
    )

def self_get_content(query, n_results=10, n_neighbors=1000):

    logger.debug("Estou em 'self_get_content'")

    result = self_predict(
        self_collection,
        query,
        n_results=n_results,
        n_neighbors=n_neighbors,
    )

    return self__set_query_content(result['documents'][0], result['metadatas'][0])

def self_handler_start_conversation(message: str, *args, **kwargs) -> str:

    logger.debug("Estou em 'self_handler_start_conversation'")

    self_chain_greeting = LLMChain(
        llm=self_llm_chat, prompt=self_prompt_greeting, verbose=self_verbose_chains, memory=kwargs["memory"]
    )

    return self_chain_greeting.predict(history=kwargs["history"], human_input=message)

def self_handler_query(message: str, *args, **kwargs) -> str:

    logger.debug("Estou em 'self_handler_query'")

    self_chain_query = LLMChain(
        llm=self_llm_chat, prompt=self_prompt_query, verbose=self_verbose_chains, memory=kwargs["memory"]
    )

    context = self_get_content(query=message)

    for i in range(3):
        response = self_chain_query.predict(history=kwargs["history"], human_input=message, context=context)
        resp_dict = extract_dict_from_string(response)
        if len(resp_dict) > 0:
            return (
                f'resposta: {resp_dict["resposta"]}\n\n'
                f'link_noticia: {resp_dict["link"]}\n\n'
                f'data_noticia: {resp_dict["data"]}\n\n'
                f'titulo_noticia: {resp_dict["titulo"]}\n\n'
                f'autor_noticia: {resp_dict["autor"]}\n\n'
            )
        continue
        
    return response
    

def self_handler_fallback(*args, **kwargs) -> str:

    logger.debug("Estou em 'self_handler_fallback'")

    return (
        "Desculpe, mas não posso responder a essa pergunta. "
        "Algo em que possa ajudar sobre notícias de Poços de Caldas e região?"
    )


# Atributes

In [103]:
self_llm = ChatOpenAI(model_name=_config.LLM_MODEL_NAME, temperature=0)
self_llm_chat = ChatOpenAI(model_name=_config.LLM_MODEL_NAME, temperature=0, model_kwargs={"stop": ["HUMAN_INPUT", "IA:"]})


self_embeddings = HuggingFaceEmbeddings(model_name=_config.HUGGINGFACE_EMBEDDING_MODEL_NAME)
self_embedder = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=_config.HUGGINGFACE_EMBEDDING_MODEL_NAME)

self_verbose_chains = True

# Load utilitary prompts and chains
self_prompt_intention = get_prompt("prompt_user_intention")
self_prompt_standalone_question = get_prompt("prompt_standalone_question")

# Load chat prompts and chains
self_prompt_greeting = get_prompt("prompt_greeting")
self_prompt_query = get_prompt("prompt_query")

self_collection = self_get_chroma_collection()

self_memory = ConversationSummaryBufferMemory(
    llm=OpenAI(temperature=0),
    chat_history=ChatMessageHistory(),
    return_messages=True,
    memory_key="chat_history",
    input_key="human_input",
    human_prefix="Human",
    ai_prefix="AI"
)

No sentence-transformers model found with name /home/jovyan/.cache/torch/sentence_transformers/unicamp-dl_ptt5-base-portuguese-vocab. Creating a new one with MEAN pooling.
2024-01-05 03:19:07.554 | DEBUG    | __main__:get_prompt:3 - Estou em 'get_prompt'
2024-01-05 03:19:07.555 | DEBUG    | __main__:get_prompt:3 - Estou em 'get_prompt'
2024-01-05 03:19:07.556 | DEBUG    | __main__:get_prompt:3 - Estou em 'get_prompt'
2024-01-05 03:19:07.557 | DEBUG    | __main__:get_prompt:3 - Estou em 'get_prompt'
2024-01-05 03:19:07.558 | DEBUG    | __main__:self_get_chroma_collection:12 - Estou em 'self_get_chroma_collection'


# Execute

In [104]:
message = "Qual a última notícia de acidentes no bairro Dom Bosco?"

In [105]:
chat_history = "".join(self_format_history_message(self_memory.chat_memory.messages))
chat_history

2024-01-05 03:19:07.618 | DEBUG    | __main__:self_format_history_message:27 - Estou em 'self_format_history_message'


''

## Melhorar query

In [106]:
standalone_question = self_get_standalone_question(message=message, history=chat_history)
logger.debug(f"Pergunta original: {message}")
logger.debug(f"Pergunta melhorada: {standalone_question}")

2024-01-05 03:19:07.626 | DEBUG    | __main__:self_get_standalone_question:37 - Estou em 'self_get_standalone_question'




> Entering new LLMChain chain...
Prompt after formatting:
Voce vai receber uma pergunta de follow up do usuario, que pode nao ter muitos detalhes.
O usuario quer realizar consultas na base de dados que contem noticias de Pocos de Caldas e regiao.
REESCREVA a frase do HUMAN INPUT para que ela possa ser considerada uma pergunta independente.
Considere o CHAT HISTORY para manter o contexto.
NAO RESPONDA AO USUARIO, APENAS REESCREVA A PERGUNTA.
Sua resposta deve conter apenas a pergunta reescrita.

## CHAT HISTORY


## HUMAN INPUT
Qual a última notícia de acidentes no bairro Dom Bosco?

IA:


2024-01-05 03:19:08.928 | DEBUG    | __main__:<module>:2 - Pergunta original: Qual a última notícia de acidentes no bairro Dom Bosco?
2024-01-05 03:19:08.930 | DEBUG    | __main__:<module>:3 - Pergunta melhorada: Qual é a última notícia de acidentes no bairro Dom Bosco?



> Finished chain.


## Identificar intenção

In [107]:
intention = self_get_user_intention(message=standalone_question, history=chat_history)
logger.debug(f"Intenção: {intention}")

2024-01-05 03:19:08.940 | DEBUG    | __main__:self_get_user_intention:47 - Estou em 'self_get_user_intention'




> Entering new LLMChain chain...
Prompt after formatting:
Analise a mensagem do usuario em "HUMAN INPUT" e o "CHAT HISTORY" e classifique a intencao de acordo com as seguintes categorias:

- "Inicio de conversa": Saudacoes e cumprimentos do usuario, reconheca frases como "oi", "ola", "bom dia", "tudo bem:", etc.
- "Consulta de conteudo": Perguntas sobre noticias da base de dados, reconheca trechos como "quais sao as ultimas noticias", "o que foi reportado no bairro", etc.
- "": QUALQUER PERGUNTA QUE NAO SE ENCAIXE NAS CATEGORIAS ACIMA, ou que nao possa ser respondida com o contexto disponivel na base de dados, a qual contem noticias de Pocos de Caldas e regiao.

Escolha apenas uma das opcoes. Sua resposta deve conter APENAS a categoria.

## CHAT HISTORY


## HUMAN INPUT
Qual é a última notícia de acidentes no bairro Dom Bosco?

IA:


2024-01-05 03:19:09.607 | DEBUG    | __main__:<module>:2 - Intenção: consulta de conteudo



> Finished chain.


## Handlers

In [108]:
handlers = {
    "inicio de conversa": self_handler_start_conversation,
    "consulta de conteudo": self_handler_query,
    "": self_handler_fallback
}


response = handlers[intention](
    message=message,
    memory=self_memory,
    history=chat_history
)

logger.debug(f"Response: {response}")

2024-01-05 03:19:09.611 | DEBUG    | __main__:self_handler_query:153 - Estou em 'self_handler_query'
2024-01-05 03:19:09.613 | DEBUG    | __main__:self_get_content:130 - Estou em 'self_get_content'
2024-01-05 03:19:09.613 | DEBUG    | __main__:self_predict:75 - Estou em 'self_predict'
2024-01-05 03:19:09.973 | DEBUG    | __main__:self__set_query_content:107 - Estou em 'self__set_query_content'
2024-01-05 03:19:09.973 | DEBUG    | __main__:self__set_local_context:119 - Estou em 'self__set_local_context'
2024-01-05 03:19:09.974 | DEBUG    | __main__:self__set_local_context:119 - Estou em 'self__set_local_context'
2024-01-05 03:19:09.974 | DEBUG    | __main__:self__set_local_context:119 - Estou em 'self__set_local_context'
2024-01-05 03:19:09.974 | DEBUG    | __main__:self__set_local_context:119 - Estou em 'self__set_local_context'
2024-01-05 03:19:09.975 | DEBUG    | __main__:self__set_local_context:119 - Estou em 'self__set_local_context'
2024-01-05 03:19:09.975 | DEBUG    | __main__:se



> Entering new LLMChain chain...
Prompt after formatting:
Voce e um chatbot que responde sobre noticias da cidade de Pocos de Caldas.

Sua tarefa e responder a mensagem do usuário em HUMAN INPUT utilizando o CONTEXTO providenciado.
Considere o CHAT HISTORY.
Nao responda nada que nao esteja no contexto. Se o contexto nao tiver informacoes relevantes a pergunta do usuario, responda "nao sei".
Nao considere nenhuma noticia que nao seja de Pocos de Caldas.

Formate sua resposta como um JSON conforme indicado abaxo:
{
    "resposta": "sua resposta aqui",
    "link": "link da noticia que voce usou para elaborar sua resposta",
    "data": "data da noticia",
    "titulo": "titulo da noticia",
    "autor": "autor da noticia"
}
Nao responda nada alem do JSON.

## CONTEXTO
None

## CHAT HISTORY


## HUMAN INPUT
Qual a última notícia de acidentes no bairro Dom Bosco?

IA:


2024-01-05 03:19:10.720 | DEBUG    | __main__:extract_dict_from_string:58 - Estou em 'extract_dict_from_string'



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Voce e um chatbot que responde sobre noticias da cidade de Pocos de Caldas.

Sua tarefa e responder a mensagem do usuário em HUMAN INPUT utilizando o CONTEXTO providenciado.
Considere o CHAT HISTORY.
Nao responda nada que nao esteja no contexto. Se o contexto nao tiver informacoes relevantes a pergunta do usuario, responda "nao sei".
Nao considere nenhuma noticia que nao seja de Pocos de Caldas.

Formate sua resposta como um JSON conforme indicado abaxo:
{
    "resposta": "sua resposta aqui",
    "link": "link da noticia que voce usou para elaborar sua resposta",
    "data": "data da noticia",
    "titulo": "titulo da noticia",
    "autor": "autor da noticia"
}
Nao responda nada alem do JSON.

## CONTEXTO
None

## CHAT HISTORY


## HUMAN INPUT
Qual a última notícia de acidentes no bairro Dom Bosco?

IA:


2024-01-05 03:19:11.181 | DEBUG    | __main__:extract_dict_from_string:58 - Estou em 'extract_dict_from_string'



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Voce e um chatbot que responde sobre noticias da cidade de Pocos de Caldas.

Sua tarefa e responder a mensagem do usuário em HUMAN INPUT utilizando o CONTEXTO providenciado.
Considere o CHAT HISTORY.
Nao responda nada que nao esteja no contexto. Se o contexto nao tiver informacoes relevantes a pergunta do usuario, responda "nao sei".
Nao considere nenhuma noticia que nao seja de Pocos de Caldas.

Formate sua resposta como um JSON conforme indicado abaxo:
{
    "resposta": "sua resposta aqui",
    "link": "link da noticia que voce usou para elaborar sua resposta",
    "data": "data da noticia",
    "titulo": "titulo da noticia",
    "autor": "autor da noticia"
}
Nao responda nada alem do JSON.

## CONTEXTO
None

## CHAT HISTORY


## HUMAN INPUT
Qual a última notícia de acidentes no bairro Dom Bosco?

IA:


2024-01-05 03:19:11.591 | DEBUG    | __main__:extract_dict_from_string:58 - Estou em 'extract_dict_from_string'
2024-01-05 03:19:11.592 | DEBUG    | __main__:<module>:14 - Response: Não sei.



> Finished chain.
